In [67]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [68]:
# os에 따라 폰트 설정하기
import os

if os.name == 'nt':
    font_family = "Malgun Gothic"
else:
    font_family = "AppleGothic"
    
sns.set(font=font_family, rc={"axes.unicode_minus" : False })

In [69]:
df = pd.read_csv("weather.csv")
df.shape

(7665, 62)

In [70]:
df.head()

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),강수 계속시간(hr),10분 최다 강수량(mm),...,1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr),합계 일조시간(hr),합계 일사량(MJ/m2)
0,108,서울,2017-01-01,2.7,-1.6,540.0,6.9,1419.0,NaN,NaN,...,7.1,10.2,15.7,17.5,0.7,1.0,0.3,NaN,NaN,NaN
1,108,서울,2017-01-02,5.0,1.8,2355.0,9.2,1355.0,2.08,NaN,...,7.0,10.1,15.6,17.5,0.9,1.2,NaN,NaN,NaN,NaN
2,108,서울,2017-01-03,2.0,-2.3,731.0,7.7,1444.0,NaN,NaN,...,6.9,9.9,15.5,17.4,1.3,1.8,NaN,NaN,NaN,NaN
3,108,서울,2017-01-04,3.9,1.0,1.0,8.9,1519.0,NaN,NaN,...,6.9,9.8,15.4,17.4,1.5,2.2,NaN,NaN,NaN,NaN
4,108,서울,2017-01-05,3.8,-0.1,316.0,7.3,1444.0,NaN,NaN,...,6.9,9.7,15.4,17.4,1.6,2.3,NaN,NaN,NaN,NaN


### 학습, 예측 데이터셋 만들기

In [71]:
# 예측할 정답값
label_name = '평균기온(°C)'

In [72]:
# 결측치를 0값으로 임의로 채움
df = df.fillna(0)

In [73]:
# 수치데이터만 feature 로 사용
num_desc = df.describe()
feature_name = num_desc.columns.tolist()
feature_name

['지점',
 '평균기온(°C)',
 '최저기온(°C)',
 '최저기온 시각(hhmi)',
 '최고기온(°C)',
 '최고기온 시각(hhmi)',
 '강수 계속시간(hr)',
 '10분 최다 강수량(mm)',
 '10분 최다강수량 시각(hhmi)',
 '1시간 최다강수량(mm)',
 '1시간 최다 강수량 시각(hhmi)',
 '일강수량(mm)',
 '최대 순간 풍속(m/s)',
 '최대 순간 풍속 풍향(16방위)',
 '최대 순간풍속 시각(hhmi)',
 '최대 풍속(m/s)',
 '최대 풍속 풍향(16방위)',
 '최대 풍속 시각(hhmi)',
 '평균 풍속(m/s)',
 '풍정합(100m)',
 '평균 이슬점온도(°C)',
 '최소 상대습도(%)',
 '최소 상대습도 시각(hhmi)',
 '평균 상대습도(%)',
 '평균 증기압(hPa)',
 '평균 현지기압(hPa)',
 '최고 해면기압(hPa)',
 '최고 해면기압 시각(hhmi)',
 '최저 해면기압(hPa)',
 '최저 해면기압 시각(hhmi)',
 '평균 해면기압(hPa)',
 '가조시간(hr)',
 '합계 일조 시간(hr)',
 '1시간 최다일사 시각(hhmi)',
 '1시간 최다일사량(MJ/m2)',
 '합계 일사(MJ/m2)',
 '일 최심신적설(cm)',
 '일 최심신적설 시각(hhmi)',
 '일 최심적설(cm)',
 '일 최심적설 시각(hhmi)',
 '합계 3시간 신적설(cm)',
 '평균 전운량(1/10)',
 '평균 중하층운량(1/10)',
 '평균 지면온도(°C)',
 '최저 초상온도(°C)',
 '평균 5cm 지중온도(°C)',
 '평균 10cm 지중온도(°C)',
 '평균 20cm 지중온도(°C)',
 '평균 30cm 지중온도(°C)',
 '0.5m 지중온도(°C)',
 '1.0m 지중온도(°C)',
 '1.5m 지중온도(°C)',
 '3.0m 지중온도(°C)',
 '5.0m 지중온도(°C)',
 '합계 대형증발량(mm)',
 '합계 소형증발량(mm)',
 '9-9강수(mm)'

In [74]:
# feature와 label 값으로 나눔
X = df[feature_name].drop([label_name], axis=1)
y = df[label_name]

X.shape, y.shape

((7665, 59), (7665,))

In [75]:
# train, test 세트로 나눔
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [76]:
# 텐서플로우 로드
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.12.0


### 텐서플로를 통한 평균기온 예측

In [77]:
model = keras.Sequential([
    layers.Dense(64, activation='swish', input_shape=[X_train.shape[1]]),
    layers.Dense(128, activation='swish'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mse',
            optimizer=optimizer,
            metrics=['mae', 'mse'])

In [78]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
161/161 [==============================] - 2s 5ms/step - loss: 10258.6973 - mae: 66.8243 - mse: 10258.6973
Epoch 2/5
161/161 [==============================] - 1s 4ms/step - loss: 244.9153 - mae: 11.4017 - mse: 244.9153
Epoch 3/5
161/161 [==============================] - 0s 3ms/step - loss: 81.5399 - mae: 7.2171 - mse: 81.5399
Epoch 4/5
161/161 [==============================] - 1s 3ms/step - loss: 55.3390 - mae: 5.9131 - mse: 55.3390
Epoch 5/5
161/161 [==============================] - 1s 3ms/step - loss: 41.6195 - mae: 5.1599 - mse: 41.6195


### 예측하기

In [79]:
y_pred = model.predict(X_test)
y_pred

80/80 [==============================] - 0s 2ms/step


array([[ 9.685562 ],
       [13.677121 ],
       [12.624794 ],
       ...,
       [23.833534 ],
       [ 7.4977975],
       [22.634932 ]], dtype=float32)

In [80]:
score = model.evaluate(X_test, y_test, verbose=2)
score

80/80 - 0s - loss: 25.3752 - mae: 4.0465 - mse: 25.3752 - 452ms/epoch - 6ms/step


[25.37517738342285, 4.046520233154297, 25.37517738342285]

In [81]:
# MAE(Mean Absolute Error) : 평균 절대 오차
MAE = score[1]
MAE

4.046520233154297

In [82]:
# MSE(Mean Square Error) : 평균 제곱 오차
MSE = score[2]
MSE

25.37517738342285

In [83]:
# RMSE(Root Mean Square Error) : 평균 제곱근 오차
RMSE = np.sqrt(MSE)
RMSE

5.0373780266546255